---

# <center> GFootball Stable-Baselines3 </center>

---
<center><img src="https://raw.githubusercontent.com/DLR-RM/stable-baselines3/master/docs/_static/img/logo.png" width="308" height="268" alt="Stable-Baselines3"></center>
<center><small>Image from Stable-Baselines3 repository</small></center>

---
This notebook uses the [Stable-Baselines3](https://github.com/DLR-RM/stable-baselines3) library to train a [PPO](https://openai.com/blog/openai-baselines-ppo/) reinforcement learning agent on [GFootball Academy](https://github.com/google-research/football/tree/master/gfootball/scenarios) scenarios, applying the architecture from the paper "[Google Research Football: A Novel Reinforcement Learning Environment](https://arxiv.org/abs/1907.11180)".

In [ ]:
%%bash
# dependencies
apt-get -y update > /dev/null
apt-get -y install libsdl2-gfx-dev libsdl2-ttf-dev > /dev/null

# cloudpickle, pytorch, gym
pip3 install "cloudpickle==1.3.0"
pip3 install "torch==1.5.1"
pip3 install "gym==0.17.2"

# gfootball
GRF_VER=v2.8
GRF_PATH=football/third_party/gfootball_engine/lib
GRF_URL=https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_${GRF_VER}.so
git clone -b ${GRF_VER} https://github.com/google-research/football.git
mkdir -p ${GRF_PATH}
wget -q ${GRF_URL} -O ${GRF_PATH}/prebuilt_gameplayfootball.so
cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install . && cd ..

# kaggle-environments
git clone https://github.com/Kaggle/kaggle-environments.git
cd kaggle-environments && pip3 install . && cd ..

# stable-baselines3
git clone https://github.com/DLR-RM/stable-baselines3.git
cd stable-baselines3 && pip3 install . && cd ..

# housekeeping
rm -rf football kaggle-environments stable-baselines3

In [ ]:
!cp "../input/gfootball-academy/visualizer.py" .

In [ ]:
import os
import base64
import pickle
import zlib
import gym
import numpy as np
import pandas as pd
import torch as th
from torch import nn, tensor
from collections import deque
from gym.spaces import Box, Discrete
from kaggle_environments import make
from kaggle_environments.envs.football.helpers import *
from gfootball.env import create_environment, observation_preprocessing
from stable_baselines3 import PPO
from stable_baselines3.ppo import CnnPolicy
from stable_baselines3.common import results_plotter
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.vec_env.dummy_vec_env import DummyVecEnv
from stable_baselines3.common.vec_env.subproc_vec_env import SubprocVecEnv
from IPython.display import HTML
from visualizer import visualize
from matplotlib import pyplot as plt
%matplotlib inline

---
# Football Gym
> [Stable-Baselines3: Custom Environments](https://stable-baselines3.readthedocs.io/en/master/guide/custom_env.html)<br/>
> [SEED RL Agent](https://www.kaggle.com/piotrstanczyk/gfootball-train-seed-rl-agent): stacked observations

In [ ]:
class FootballGym(gym.Env):
    spec = None
    metadata = None
    
    def __init__(self, config=None):
        super(FootballGym, self).__init__()
        env_name = "academy_empty_goal_close"
        rewards = "scoring,checkpoints"
        if config is not None:
            env_name = config.get("env_name", env_name)
            rewards = config.get("rewards", rewards)
        self.env = create_environment(
            env_name=env_name,
            stacked=False,
            representation="raw",
            rewards = rewards,
            write_goal_dumps=False,
            write_full_episode_dumps=False,
            render=False,
            write_video=False,
            dump_frequency=1,
            logdir=".",
            extra_players=None,
            number_of_left_players_agent_controls=1,
            number_of_right_players_agent_controls=0)  
        self.action_space = Discrete(19)
        self.observation_space = Box(low=0, high=255, shape=(72, 96, 16), dtype=np.uint8)
        self.reward_range = (-1, 1)
        self.obs_stack = deque([], maxlen=4)
        
    def transform_obs(self, raw_obs):
        obs = raw_obs[0]
        obs = observation_preprocessing.generate_smm([obs])
        if not self.obs_stack:
            self.obs_stack.extend([obs] * 4)
        else:
            self.obs_stack.append(obs)
        obs = np.concatenate(list(self.obs_stack), axis=-1)
        obs = np.squeeze(obs)
        return obs

    def reset(self):
        self.obs_stack.clear()
        obs = self.env.reset()
        obs = self.transform_obs(obs)
        return obs
    
    def step(self, action):
        obs, reward, done, info = self.env.step([action])
        obs = self.transform_obs(obs)
        return obs, float(reward), done, info
    
check_env(env=FootballGym(), warn=True)

---
# Football CNN
> [Stable-Baselines3: Custom Policy Network](https://stable-baselines3.readthedocs.io/en/master/guide/custom_policy.html)<br/>
> [Google Research Football: A Novel Reinforcement Learning Environment](https://arxiv.org/abs/1907.11180)

In [ ]:
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=True)

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.relu = nn.ReLU()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.conv2 = conv3x3(out_channels, out_channels, stride)
        
    def forward(self, x):
        residual = x
        out = self.relu(x)
        out = self.conv1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out += residual
        return out
    
class FootballCNN(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=256):
        super().__init__(observation_space, features_dim)
        in_channels = observation_space.shape[0]  # channels x height x width
        self.cnn = nn.Sequential(
            conv3x3(in_channels=in_channels, out_channels=32),
            nn.MaxPool2d(kernel_size=3, stride=2, dilation=1, ceil_mode=False),
            ResidualBlock(in_channels=32, out_channels=32),
            ResidualBlock(in_channels=32, out_channels=32),
            nn.ReLU(),
            nn.Flatten(),
        )
        self.linear = nn.Sequential(
          nn.Linear(in_features=52640, out_features=features_dim, bias=True),
          nn.ReLU(),
        )

    def forward(self, obs):
        return self.linear(self.cnn(obs))

---
# PPO Model
> [Stable-Baselines3: PPO](https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html)<br/>
> [Stable-Baselines3: Vectorized Environments](https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html)<br/>
> [Stable-Baselines3: Custom Policy Network](https://stable-baselines3.readthedocs.io/en/master/guide/custom_policy.html)<br/>
> [GFootball: A Novel Reinforcement Learning Environment](https://arxiv.org/abs/1907.11180)<br/>
> [GFootball: Academy Scenarios](https://github.com/google-research/football/tree/master/gfootball/scenarios)<br/>

In [ ]:
scenarios = {0: "academy_empty_goal_close",
             1: "academy_empty_goal",
             2: "academy_run_to_score",
             3: "academy_run_to_score_with_keeper",
             4: "academy_pass_and_shoot_with_keeper",
             5: "academy_run_pass_and_shoot_with_keeper",
             6: "academy_3_vs_1_with_keeper",
             7: "academy_corner",
             8: "academy_counterattack_easy",
             9: "academy_counterattack_hard",
             10: "academy_single_goal_versus_lazy",
             11: "11_vs_11_kaggle"}
scenario_name = scenarios[7]

In [ ]:
def make_env(config=None, rank=0):
    def _init():
        env = FootballGym(config)
        log_file = os.path.join(".", str(rank))
        env = Monitor(env, log_file, allow_early_resets=True)
        return env
    return _init

In [ ]:
n_envs = 4
config={"env_name":scenario_name}
train_env = DummyVecEnv([make_env(config, rank=i) for i in range(n_envs)])
# train_env = SubprocVecEnv([make_env(config, rank=i) for i in range(n_envs)])

n_steps = 512
policy_kwargs = dict(features_extractor_class=FootballCNN,
                     features_extractor_kwargs=dict(features_dim=256))
# model = PPO(CnnPolicy, train_env, 
#             policy_kwargs=policy_kwargs, 
#             learning_rate=0.000343, 
#             n_steps=n_steps, 
#             batch_size=8, 
#             n_epochs=2, 
#             gamma=0.993,
#             gae_lambda=0.95,
#             clip_range=0.08, 
#             ent_coef=0.003, 
#             vf_coef=0.5, 
#             max_grad_norm=0.64, 
#             verbose=0)
model = PPO.load("../input/gfootball-stable-baselines3/ppo_gfootball.zip", train_env)

---
# Training
> [Stable-Baselines3: Examples](https://stable-baselines3.readthedocs.io/en/master/guide/examples.html)<br/>
> [Stable-Baselines3: Callbacks](https://stable-baselines3.readthedocs.io/en/master/guide/callbacks.html)

In [ ]:
from tqdm.notebook import tqdm
class ProgressBar(BaseCallback):
    def __init__(self, verbose=0):
        super(ProgressBar, self).__init__(verbose)
        self.pbar = None

    def _on_training_start(self):
        factor = np.ceil(self.locals['total_timesteps'] / self.model.n_steps)
        n = 1
        try:
            n = len(self.training_env.envs)
        except AttributeError:
            try:
                n = len(self.training_env.remotes)
            except AttributeError:
                n = 1
        total = int(self.model.n_steps * factor / n)
        self.pbar = tqdm(total=total)

    def _on_rollout_start(self):
        self.pbar.refresh()

    def _on_step(self):
        self.pbar.update(1)
        return True

    def _on_rollout_end(self):
        self.pbar.refresh()

    def _on_training_end(self):
        self.pbar.close()
        self.pbar = None

progressbar = ProgressBar()

In [ ]:
total_epochs = 200
total_timesteps = n_steps * n_envs * total_epochs
model.learn(total_timesteps=total_timesteps, callback=progressbar)
model.save("ppo_gfootball")

In [ ]:
plt.style.use(['seaborn-whitegrid'])
results_plotter.plot_results(["."], total_timesteps, results_plotter.X_TIMESTEPS, "GFootball Timesteps")
results_plotter.plot_results(["."], total_timesteps, results_plotter.X_EPISODES, "GFootball Episodes")

In [ ]:
plt.style.use(['seaborn-whitegrid'])
log_files = [os.path.join(".", f"{i}.monitor.csv") for i in range(n_envs)]

nrows = np.ceil(n_envs/2)
fig = plt.figure(figsize=(8, 2 * nrows))
for i, log_file in enumerate(log_files):
    if os.path.isfile(log_file):
        df = pd.read_csv(log_file, skiprows=1)
        plt.subplot(nrows, 2, i+1, label=log_file)
        df['r'].rolling(window=100).mean().plot(title=f"Rewards: Env {i}")
        plt.tight_layout()
plt.show()

In [ ]:
model = PPO.load("ppo_gfootball")
test_env = FootballGym({"env_name":scenario_name})
obs = test_env.reset()
done = False
while not done:
    action, state = model.predict(obs, deterministic=True)
    obs, reward, done, info = test_env.step(action)
    print(f"{Action(action).name.ljust(16,' ')}\t{round(reward,2)}\t{info}")

---
# Agent
> [Stable-Baselines: Exporting Models](https://stable-baselines.readthedocs.io/en/master/guide/export.html)<br/>
> [Stable-Baselines: Converting a Model into PyTorch](https://github.com/hill-a/stable-baselines/issues/372)<br/>
> [Connect4: Make Submission with Stable-Baselines3](https://www.kaggle.com/toshikazuwatanabe/connect4-make-submission-with-stable-baselines3)

In [ ]:
%%writefile submission.py
import base64
import pickle
import zlib
import numpy as np
import torch as th
from torch import nn, tensor
from collections import deque
from gfootball.env import observation_preprocessing

state_dict = _STATE_DICT_

state_dict = pickle.loads(zlib.decompress(base64.b64decode(state_dict)))

def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=True)

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.relu = nn.ReLU()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.conv2 = conv3x3(out_channels, out_channels, stride)
        
    def forward(self, x):
        residual = x
        out = self.relu(x)
        out = self.conv1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out += residual
        return out
    
class PyTorchCnnPolicy(nn.Module):
    global state_dict
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            conv3x3(in_channels=16, out_channels=32),
            nn.MaxPool2d(kernel_size=3, stride=2, dilation=1, ceil_mode=False),
            ResidualBlock(in_channels=32, out_channels=32),
            ResidualBlock(in_channels=32, out_channels=32),
            nn.ReLU(),
            nn.Flatten(),
        )
        self.linear = nn.Sequential(
          nn.Linear(in_features=52640, out_features=256, bias=True),
          nn.ReLU(),
        )
        self.action_net = nn.Sequential(
          nn.Linear(in_features=256, out_features=19, bias=True),
          nn.ReLU(),
        )
        self.out_activ = nn.Softmax(dim=1)
        self.load_state_dict(state_dict)

    def forward(self, x):
        x = tensor(x).float() / 255.0  # normalize
        x = x.permute(0, 3, 1, 2).contiguous()  # 1 x channels x height x width
        x = self.cnn(x)
        x = self.linear(x)
        x = self.action_net(x)
        x = self.out_activ(x)
        return int(x.argmax())
    
obs_stack = deque([], maxlen=4)
def transform_obs(raw_obs):
    global obs_stack
    obs = raw_obs['players_raw'][0]
    obs = observation_preprocessing.generate_smm([obs])
    if not obs_stack:
        obs_stack.extend([obs] * 4)
    else:
        obs_stack.append(obs)
    obs = np.concatenate(list(obs_stack), axis=-1)
    return obs

policy = PyTorchCnnPolicy()
policy = policy.float().to('cpu').eval()
def agent(raw_obs):
    obs = transform_obs(raw_obs)
    action = policy(obs)
    return [action]

In [ ]:
model = PPO.load("ppo_gfootball")
_state_dict = model.policy.to('cpu').state_dict()
state_dict = {
    "cnn.0.weight":_state_dict['features_extractor.cnn.0.weight'], 
    "cnn.0.bias":_state_dict['features_extractor.cnn.0.bias'], 
    "cnn.2.conv1.weight":_state_dict['features_extractor.cnn.2.conv1.weight'], 
    "cnn.2.conv1.bias":_state_dict['features_extractor.cnn.2.conv1.bias'],
    "cnn.2.conv2.weight":_state_dict['features_extractor.cnn.2.conv2.weight'], 
    "cnn.2.conv2.bias":_state_dict['features_extractor.cnn.2.conv2.bias'], 
    "cnn.3.conv1.weight":_state_dict['features_extractor.cnn.3.conv1.weight'], 
    "cnn.3.conv1.bias":_state_dict['features_extractor.cnn.3.conv1.bias'], 
    "cnn.3.conv2.weight":_state_dict['features_extractor.cnn.3.conv2.weight'], 
    "cnn.3.conv2.bias":_state_dict['features_extractor.cnn.3.conv2.bias'], 
    "linear.0.weight":_state_dict['features_extractor.linear.0.weight'], 
    "linear.0.bias":_state_dict['features_extractor.linear.0.bias'], 
    "action_net.0.weight":_state_dict['action_net.weight'],
    "action_net.0.bias":_state_dict['action_net.bias'],
}
state_dict = base64.b64encode(zlib.compress(pickle.dumps(state_dict)))
with open('submission.py', 'r') as file:
    src = file.read()
src = src.replace("_STATE_DICT_", f"{state_dict}")
with open('submission.py', 'w') as file:
    file.write(src)

---
# Testing

In [ ]:
kaggle_env = make("football", debug = False,
                  configuration={"scenario_name": scenario_name, 
                                 "running_in_notebook": True,
                                 "save_video": False})

In [ ]:
output = kaggle_env.run(["submission.py", "do_nothing"])

In [ ]:
scores = output[-1][0]["observation"]["players_raw"][0]["score"]
print("Scores  {0} : {1}".format(*scores))
print("Rewards {0} : {1}".format(output[-1][0]["reward"], output[-1][1]["reward"]))

In [ ]:
viz = visualize(output)

> Modified [Human Readable Visualization](https://www.kaggle.com/jaronmichal/human-readable-visualization)

In [ ]:
HTML(viz.to_html5_video())

---
# Checkpoints

0. [academy_empty_goal_close @ 800K steps](https://www.kaggle.com/kwabenantim/gfootball-stable-baselines3?scriptVersionId=45569809#Test-Agent) (Nature CNN)<br/>
1. [academy_empty_goal @ 800K steps](https://www.kaggle.com/kwabenantim/gfootball-stable-baselines3?scriptVersionId=45639135#Test-Agent) (Nature CNN)<br/>
2. [academy_run_to_score @ 800K steps](https://www.kaggle.com/kwabenantim/gfootball-stable-baselines3?scriptVersionId=45941674#Test-Agent) (Nature CNN)<br/>
3. [academy_run_to_score_with_keeper @ 800K steps](https://www.kaggle.com/kwabenantim/gfootball-stable-baselines3?scriptVersionId=45703399#Test-Agent) (Nature CNN)<br/>
4. [academy_pass_and_shoot_with_keeper @ 800K steps](https://www.kaggle.com/kwabenantim/gfootball-stable-baselines3?scriptVersionId=45716494#Test-Agent) (Nature CNN)<br/>
5. [academy_run_pass_and_shoot_with_keeper @ 1.6M steps](https://www.kaggle.com/kwabenantim/gfootball-stable-baselines3?scriptVersionId=46590578#Testing) (Nature CNN)<br/>
6. [academy_3_vs_1_with_keeper @ 500K steps](https://www.kaggle.com/kwabenantim/gfootball-stable-baselines3?scriptVersionId=46843278#Testing) (GFootball CNN)